In [1]:
# prepare train dataset without transactions.csv

In [2]:
import numpy as np
import pandas as pd
import feather
from tqdm import tqdm
from joblib import Parallel, delayed

In [3]:
# save the file as each step
df_train3 = feather.read_dataframe('../cache/train3.feather')

In [4]:
df_train3.shape

(125497040, 14)

In [5]:
df_train3.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable,city,state,store_type,cluster,dcoilwtico
0,0,2013-01-01,25,103665,7.0,Unk,BREAD/BAKERY,2712,1,Salinas,Santa Elena,D,1,93.14
1,1,2013-01-01,25,105574,1.0,Unk,GROCERY I,1045,0,Salinas,Santa Elena,D,1,93.14
2,2,2013-01-01,25,105575,2.0,Unk,GROCERY I,1045,0,Salinas,Santa Elena,D,1,93.14
3,3,2013-01-01,25,108079,1.0,Unk,GROCERY I,1030,0,Salinas,Santa Elena,D,1,93.14
4,4,2013-01-01,25,108701,1.0,Unk,DELI,2644,1,Salinas,Santa Elena,D,1,93.14


In [36]:
df_h = pd.read_csv('../data/holidays_events.csv')

In [37]:
df_h.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [38]:
df_h.shape

(350, 6)

In [39]:
len(set(df_h.date))

312

In [40]:
len(set(df_train3.date))

1684

In [41]:
len(set(df_train3.date) - set(df_h.date))

1432

In [42]:
len(set(df_h.date) - set(df_train3.date))

60

In [43]:
set(df_h.date) - set(df_train3.date)

{'2012-03-02',
 '2012-04-01',
 '2012-04-12',
 '2012-04-14',
 '2012-04-21',
 '2012-05-12',
 '2012-06-23',
 '2012-06-25',
 '2012-07-03',
 '2012-07-23',
 '2012-08-05',
 '2012-08-10',
 '2012-08-15',
 '2012-08-24',
 '2012-09-28',
 '2012-10-07',
 '2012-10-09',
 '2012-10-12',
 '2012-11-02',
 '2012-11-03',
 '2012-11-06',
 '2012-11-07',
 '2012-11-10',
 '2012-11-11',
 '2012-11-12',
 '2012-12-05',
 '2012-12-06',
 '2012-12-08',
 '2012-12-21',
 '2012-12-22',
 '2012-12-23',
 '2012-12-24',
 '2012-12-25',
 '2012-12-26',
 '2012-12-31',
 '2013-12-25',
 '2014-12-25',
 '2015-12-25',
 '2016-12-25',
 '2017-08-24',
 '2017-09-28',
 '2017-09-29',
 '2017-10-07',
 '2017-10-09',
 '2017-11-02',
 '2017-11-03',
 '2017-11-06',
 '2017-11-07',
 '2017-11-10',
 '2017-11-11',
 '2017-11-12',
 '2017-12-05',
 '2017-12-06',
 '2017-12-08',
 '2017-12-21',
 '2017-12-22',
 '2017-12-23',
 '2017-12-24',
 '2017-12-25',
 '2017-12-26'}

In [44]:
df_h = df_h[(df_h.date >= '2013-01-01') & (df_h.date <= '2017-08-15')]

In [45]:
df_h.shape

(286, 6)

In [46]:
set(df_h.date) - set(df_train3.date)

{'2013-12-25', '2014-12-25', '2015-12-25', '2016-12-25'}

In [47]:
df_h = df_h[(df_h.date != '2013-12-25') & (df_h.date != '2014-12-25') & (df_h.date != '2015-12-25') & 
            (df_h.date != '2016-12-25')]

In [48]:
df_h = df_h.reset_index(drop=True)

In [49]:
df_h.head()

,date,type,locale,locale_name,description,transferred
0,2013-01-01,Holiday,National,Ecuador,Primer dia del ano,False
1,2013-01-05,Work Day,National,Ecuador,Recupero puente Navidad,False
2,2013-01-12,Work Day,National,Ecuador,Recupero puente primer dia del ano,False
3,2013-02-11,Holiday,National,Ecuador,Carnaval,False
4,2013-02-12,Holiday,National,Ecuador,Carnaval,False


In [50]:
df_h.shape

(282, 6)

In [51]:
pd.value_counts(df_h.type)

Holiday       171
Event          56
Additional     38
Transfer        9
Work Day        5
Bridge          3
Name: type, dtype: int64

In [52]:
pd.value_counts(df_h.locale)

National    147
Local       117
Regional     18
Name: locale, dtype: int64

In [53]:
pd.value_counts(df_h.transferred)

False    273
True       9
Name: transferred, dtype: int64

In [54]:
# 2013-01-05 - Saturday
# 2013-01-12 - Saturday
# 2014-12-20 - Saturday
# 2015-01-10 - Saturday
# 2016-11-12 - Saturday
df_h[df_h.type == 'Work Day']

,date,type,locale,locale_name,description,transferred
1,2013-01-05,Work Day,National,Ecuador,Recupero puente Navidad,False
2,2013-01-12,Work Day,National,Ecuador,Recupero puente primer dia del ano,False
107,2014-12-20,Work Day,National,Ecuador,Recupero Puente Navidad,False
118,2015-01-10,Work Day,National,Ecuador,Recupero Puente Primer dia del ano,False
239,2016-11-12,Work Day,National,Ecuador,Recupero Puente Dia de Difuntos,False


In [55]:
df_h[(df_h.type == 'Holiday') & (df_h.transferred == True)]

,date,type,locale,locale_name,description,transferred
31,2013-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,True
93,2014-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,True
211,2016-05-24,Holiday,National,Ecuador,Batalla de Pichincha,True
222,2016-07-25,Holiday,Local,Guayaquil,Fundacion de Guayaquil,True
224,2016-08-10,Holiday,National,Ecuador,Primer Grito de Independencia,True
252,2017-01-01,Holiday,National,Ecuador,Primer dia del ano,True
258,2017-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,True
267,2017-05-24,Holiday,National,Ecuador,Batalla de Pichincha,True
279,2017-08-10,Holiday,National,Ecuador,Primer Grito de Independencia,True


In [56]:
df_h.loc[0]

date                   2013-01-01
type                      Holiday
locale                   National
locale_name               Ecuador
description    Primer dia del ano
transferred                 False
Name: 0, dtype: object

In [57]:
df_h.type[(df_h.type == 'Holiday') & (df_h.transferred == True)] = 'Work Day'

/home/watts/anaconda3/envs/rss/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [58]:
df_h[(df_h.type == 'Holiday') & (df_h.transferred == True)]

,date,type,locale,locale_name,description,transferred


In [59]:
df_h_national = df_h[df_h.locale == 'National']

In [60]:
df_h_national.shape

(147, 6)

In [61]:
df_h_national = df_h_national.drop_duplicates(['date'], keep='first')

In [62]:
df_h_national.shape

(143, 6)

In [63]:
pd.value_counts(df_h_national.type)

Event         53
Holiday       40
Additional    28
Work Day      12
Transfer       7
Bridge         3
Name: type, dtype: int64

In [64]:
df_h_national.type[(df_h_national.type != 'Event') & (df_h_national.type != 'Work Day')] = 'Holiday'

In [65]:
pd.value_counts(df_h_national.type)

Holiday     78
Event       53
Work Day    12
Name: type, dtype: int64

In [66]:
df_h_national.head()

,date,type,locale,locale_name,description,transferred
0,2013-01-01,Holiday,National,Ecuador,Primer dia del ano,False
1,2013-01-05,Work Day,National,Ecuador,Recupero puente Navidad,False
2,2013-01-12,Work Day,National,Ecuador,Recupero puente primer dia del ano,False
3,2013-02-11,Holiday,National,Ecuador,Carnaval,False
4,2013-02-12,Holiday,National,Ecuador,Carnaval,False


In [67]:
df_h_national = df_h_national.drop(['transferred', 'locale', 'description', 'locale_name'], axis=1)

In [68]:
df_h_national.head()

,date,type
0,2013-01-01,Holiday
1,2013-01-05,Work Day
2,2013-01-12,Work Day
3,2013-02-11,Holiday
4,2013-02-12,Holiday


In [69]:
df_train3['d1'] = df_train3['date'].values

In [70]:
df_train3['d1'] = (pd.to_datetime(df_train3['d1']))

In [71]:
df_train3.tail()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable,city,state,store_type,cluster,dcoilwtico,d1
125497035,125497035,2017-08-15,54,2089339,4.0,False,GROCERY I,1006,0,El Carmen,Manabi,C,3,47.57,2017-08-15
125497036,125497036,2017-08-15,54,2106464,1.0,True,BEVERAGES,1148,0,El Carmen,Manabi,C,3,47.57,2017-08-15
125497037,125497037,2017-08-15,54,2110456,192.0,False,BEVERAGES,1120,0,El Carmen,Manabi,C,3,47.57,2017-08-15
125497038,125497038,2017-08-15,54,2113914,198.0,True,CLEANING,3040,0,El Carmen,Manabi,C,3,47.57,2017-08-15
125497039,125497039,2017-08-15,54,2116416,2.0,False,GROCERY I,1060,0,El Carmen,Manabi,C,3,47.57,2017-08-15


In [72]:
df_train3["dom"] = df_train3["d1"].apply(lambda x: x.day)
df_train3["mon"] = df_train3["d1"].apply(lambda x: x.month)
df_train3["dow"] = df_train3["d1"].apply(lambda x: x.weekday())
df_train3["doy"] = df_train3["d1"].apply(lambda x: x.timetuple().tm_yday)
df_train3["dcount"] = df_train3["d1"].apply(lambda x: x.toordinal())

In [73]:
df_train3 = df_train3.drop('d1', axis=1)

In [74]:
df_train3.tail()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable,city,state,store_type,cluster,dcoilwtico,dom,mon,dow,doy,dcount
125497035,125497035,2017-08-15,54,2089339,4.0,False,GROCERY I,1006,0,El Carmen,Manabi,C,3,47.57,15,8,1,227,736556
125497036,125497036,2017-08-15,54,2106464,1.0,True,BEVERAGES,1148,0,El Carmen,Manabi,C,3,47.57,15,8,1,227,736556
125497037,125497037,2017-08-15,54,2110456,192.0,False,BEVERAGES,1120,0,El Carmen,Manabi,C,3,47.57,15,8,1,227,736556
125497038,125497038,2017-08-15,54,2113914,198.0,True,CLEANING,3040,0,El Carmen,Manabi,C,3,47.57,15,8,1,227,736556
125497039,125497039,2017-08-15,54,2116416,2.0,False,GROCERY I,1060,0,El Carmen,Manabi,C,3,47.57,15,8,1,227,736556


In [75]:
np.min(df_train3['dow'])

0

In [76]:
np.max(df_train3['dow'])

6

In [77]:
df_h_national.head()

,date,type
0,2013-01-01,Holiday
1,2013-01-05,Work Day
2,2013-01-12,Work Day
3,2013-02-11,Holiday
4,2013-02-12,Holiday


In [78]:
df_train4 = pd.merge(df_train3, df_h_national, how='left', on='date')

In [79]:
df_train4.shape

(125497040, 20)

In [80]:
df_train3.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable,city,state,store_type,cluster,dcoilwtico,dom,mon,dow,doy,dcount
0,0,2013-01-01,25,103665,7.0,Unk,BREAD/BAKERY,2712,1,Salinas,Santa Elena,D,1,93.14,1,1,1,1,734869
1,1,2013-01-01,25,105574,1.0,Unk,GROCERY I,1045,0,Salinas,Santa Elena,D,1,93.14,1,1,1,1,734869
2,2,2013-01-01,25,105575,2.0,Unk,GROCERY I,1045,0,Salinas,Santa Elena,D,1,93.14,1,1,1,1,734869
3,3,2013-01-01,25,108079,1.0,Unk,GROCERY I,1030,0,Salinas,Santa Elena,D,1,93.14,1,1,1,1,734869
4,4,2013-01-01,25,108701,1.0,Unk,DELI,2644,1,Salinas,Santa Elena,D,1,93.14,1,1,1,1,734869


In [81]:
df_train3.shape

(125497040, 19)

In [82]:
df_train4.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable,city,state,store_type,cluster,dcoilwtico,dom,mon,dow,doy,dcount,type
0,0,2013-01-01,25,103665,7.0,Unk,BREAD/BAKERY,2712,1,Salinas,Santa Elena,D,1,93.14,1,1,1,1,734869,Holiday
1,1,2013-01-01,25,105574,1.0,Unk,GROCERY I,1045,0,Salinas,Santa Elena,D,1,93.14,1,1,1,1,734869,Holiday
2,2,2013-01-01,25,105575,2.0,Unk,GROCERY I,1045,0,Salinas,Santa Elena,D,1,93.14,1,1,1,1,734869,Holiday
3,3,2013-01-01,25,108079,1.0,Unk,GROCERY I,1030,0,Salinas,Santa Elena,D,1,93.14,1,1,1,1,734869,Holiday
4,4,2013-01-01,25,108701,1.0,Unk,DELI,2644,1,Salinas,Santa Elena,D,1,93.14,1,1,1,1,734869,Holiday


In [83]:
df_train4.isnull().sum()
# type            114412272

id                     0
date                   0
store_nbr              0
item_nbr               0
unit_sales             0
onpromotion            0
family                 0
class                  0
perishable             0
city                   0
state                  0
store_type             0
cluster                0
dcoilwtico             0
dom                    0
mon                    0
dow                    0
doy                    0
dcount                 0
type           114412272
dtype: int64

In [84]:
df_train4[df_train4.type.isnull()].shape

(114412272, 20)

In [85]:
df_train4.type[(df_train4.type.isnull()) & (df_train4.dow == 5)] = 'Holiday'

/home/watts/anaconda3/envs/rss/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [86]:
df_train4.type[(df_train4.type.isnull()) & (df_train4.dow == 6)] = 'Holiday'

/home/watts/anaconda3/envs/rss/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [87]:
df_train4.isnull().sum()

id                    0
date                  0
store_nbr             0
item_nbr              0
unit_sales            0
onpromotion           0
family                0
class                 0
perishable            0
city                  0
state                 0
store_type            0
cluster               0
dcoilwtico            0
dom                   0
mon                   0
dow                   0
doy                   0
dcount                0
type           79903107
dtype: int64

In [88]:
df_train4.fillna('Work Day', inplace=True)

In [89]:
df_train4.isnull().sum()

id             0
date           0
store_nbr      0
item_nbr       0
unit_sales     0
onpromotion    0
family         0
class          0
perishable     0
city           0
state          0
store_type     0
cluster        0
dcoilwtico     0
dom            0
mon            0
dow            0
doy            0
dcount         0
type           0
dtype: int64

In [90]:
df_train4.shape

(125497040, 20)

In [91]:
df_h_national = df_h[df_h.locale == 'National']

In [92]:
df_h_national.shape

(147, 6)

In [93]:
df_h_national = df_h_national.drop_duplicates(['date'], keep='first')

In [94]:
df_h_national.shape

(143, 6)

In [95]:
df_h_national.type[(df_h_national.type != 'Event') & (df_h_national.type != 'Work Day')] = 'Holiday'

In [96]:
df_h_national.head()

,date,type,locale,locale_name,description,transferred
0,2013-01-01,Holiday,National,Ecuador,Primer dia del ano,False
1,2013-01-05,Work Day,National,Ecuador,Recupero puente Navidad,False
2,2013-01-12,Work Day,National,Ecuador,Recupero puente primer dia del ano,False
3,2013-02-11,Holiday,National,Ecuador,Carnaval,False
4,2013-02-12,Holiday,National,Ecuador,Carnaval,False


In [97]:
df_h_national = df_h_national.drop(['transferred', 'locale', 'type', 'locale_name'], axis=1)

In [98]:
df_h_national.head()

,date,description
0,2013-01-01,Primer dia del ano
1,2013-01-05,Recupero puente Navidad
2,2013-01-12,Recupero puente primer dia del ano
3,2013-02-11,Carnaval
4,2013-02-12,Carnaval


In [99]:
df_train5 = pd.merge(df_train4, df_h_national, how='left', on='date')

In [100]:
df_train5.shape

(125497040, 21)

In [101]:
df_train5.isnull().sum()

id                     0
date                   0
store_nbr              0
item_nbr               0
unit_sales             0
onpromotion            0
family                 0
class                  0
perishable             0
city                   0
state                  0
store_type             0
cluster                0
dcoilwtico             0
dom                    0
mon                    0
dow                    0
doy                    0
dcount                 0
type                   0
description    114412272
dtype: int64

In [102]:
df_train5.type[(df_train5.description.isnull()) & (df_train5.dow == 5)] = 'Weekend'

/home/watts/anaconda3/envs/rss/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [103]:
df_train5.type[(df_train5.description.isnull()) & (df_train5.dow == 6)] = 'Weekend'

/home/watts/anaconda3/envs/rss/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [104]:
df_train5.fillna('Normal', inplace=True)

In [105]:
df_train5.isnull().sum()

id             0
date           0
store_nbr      0
item_nbr       0
unit_sales     0
onpromotion    0
family         0
class          0
perishable     0
city           0
state          0
store_type     0
cluster        0
dcoilwtico     0
dom            0
mon            0
dow            0
doy            0
dcount         0
type           0
description    0
dtype: int64

In [106]:
df_train5.shape

(125497040, 21)

In [107]:
df_train5.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable,city,...,store_type,cluster,dcoilwtico,dom,mon,dow,doy,dcount,type,description
0,0,2013-01-01,25,103665,7.0,Unk,BREAD/BAKERY,2712,1,Salinas,...,D,1,93.14,1,1,1,1,734869,Holiday,Primer dia del ano
1,1,2013-01-01,25,105574,1.0,Unk,GROCERY I,1045,0,Salinas,...,D,1,93.14,1,1,1,1,734869,Holiday,Primer dia del ano
2,2,2013-01-01,25,105575,2.0,Unk,GROCERY I,1045,0,Salinas,...,D,1,93.14,1,1,1,1,734869,Holiday,Primer dia del ano
3,3,2013-01-01,25,108079,1.0,Unk,GROCERY I,1030,0,Salinas,...,D,1,93.14,1,1,1,1,734869,Holiday,Primer dia del ano
4,4,2013-01-01,25,108701,1.0,Unk,DELI,2644,1,Salinas,...,D,1,93.14,1,1,1,1,734869,Holiday,Primer dia del ano


In [108]:
feather.write_dataframe(df_train5, '../cache/train5_nt.feather') # nt for no transaction

In [3]:
df_train = feather.read_dataframe('../cache/train5_nt.feather')

In [7]:
df_train.shape

(125497040, 21)

In [8]:
df_train[df_train.date == '2014-12-25']

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable,city,...,store_type,cluster,dcoilwtico,dom,mon,dow,doy,dcount,type,description


In [15]:
df_train.tail()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable,city,...,store_type,cluster,dcoilwtico,dom,mon,dow,doy,dcount,type,description
125497035,125497035,2017-08-15,54,2089339,4.0,False,GROCERY I,1006,0,El Carmen,...,C,3,47.57,15,8,1,227,736556,Work Day,Normal
125497036,125497036,2017-08-15,54,2106464,1.0,True,BEVERAGES,1148,0,El Carmen,...,C,3,47.57,15,8,1,227,736556,Work Day,Normal
125497037,125497037,2017-08-15,54,2110456,192.0,False,BEVERAGES,1120,0,El Carmen,...,C,3,47.57,15,8,1,227,736556,Work Day,Normal
125497038,125497038,2017-08-15,54,2113914,198.0,True,CLEANING,3040,0,El Carmen,...,C,3,47.57,15,8,1,227,736556,Work Day,Normal
125497039,125497039,2017-08-15,54,2116416,2.0,False,GROCERY I,1060,0,El Carmen,...,C,3,47.57,15,8,1,227,736556,Work Day,Normal


In [16]:
df_h = pd.read_csv('../data/holidays_events.csv')

In [17]:
df_h_local = df_h[df_h.locale == 'Local']

In [18]:
df_h_local.shape

(152, 6)

In [19]:
len(set(df_h_local.locale_name))

19

In [20]:
len(set(df_train.city))

22

In [21]:
set(df_train.city) - set(df_h_local.locale_name)

{'Babahoyo', 'Daule', 'Playas'}

In [22]:
df_h_local = df_h_local.drop_duplicates(['date'], keep='first')

In [23]:
df_h_local.shape

(138, 6)

In [24]:
df_h_local.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False
5,2012-05-12,Holiday,Local,Puyo,Cantonizacion del Puyo,False


In [25]:
df_h_l1 = df_h_local.set_index('date')

In [26]:
import os
x = {i for i in range(20)}
os.sched_setaffinity(0, x)

In [27]:
def set_train_type_desc_local(date):
    t = df_h_l1.loc[date].type
    d = df_h_l1.loc[date].description
    c = df_h_l1.loc[date].locale_name

    df_train.type[(df_train.date == date) & (df_train.city == c)] = t
    df_train.description[(df_train.date == date) & (df_train.city == c)] = d

In [28]:
for date in tqdm(df_h_local.date):
    set_train_type_desc_local(date) 

  0%|          | 0/138 [00:00<?, ?it/s]/home/watts/anaconda3/envs/rss/lib/python3.5/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/watts/anaconda3/envs/rss/lib/python3.5/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
100%|██████████| 138/138 [2:00:02<00:00, 52.19s/it] 


In [29]:
df_h_regional = df_h[df_h.locale == 'Regional']

In [30]:
df_h_regional.shape

(24, 6)

In [31]:
len(set(df_h_regional.locale_name))

4

In [32]:
len(set(df_train.state))

16

In [33]:
set(df_train.state) - set(df_h_regional.locale_name)

{'Azuay',
 'Bolivar',
 'Chimborazo',
 'El Oro',
 'Esmeraldas',
 'Guayas',
 'Loja',
 'Los Rios',
 'Manabi',
 'Pastaza',
 'Pichincha',
 'Tungurahua'}

In [34]:
len(set(df_train.state) - set(df_h_regional.locale_name))

12

In [35]:
df_h_regional = df_h_regional.drop_duplicates(['date'], keep='first')

In [36]:
df_h_regional.shape

(24, 6)

In [37]:
df_h_regional.head()

,date,type,locale,locale_name,description,transferred
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
7,2012-06-25,Holiday,Regional,Imbabura,Provincializacion de Imbabura,False
23,2012-11-06,Holiday,Regional,Santo Domingo de los Tsachilas,Provincializacion de Santo Domingo,False
24,2012-11-07,Holiday,Regional,Santa Elena,Provincializacion Santa Elena,False
47,2013-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False


In [38]:
df_h_r1 = df_h_regional.set_index('date')

In [39]:
def set_train_type_desc_regional(date):
    t = df_h_r1.loc[date].type
    d = df_h_r1.loc[date].description
    s = df_h_r1.loc[date].locale_name

    df_train.type[(df_train.date == date) & (df_train.state == s)] = t
    df_train.description[(df_train.date == date) & (df_train.state == s)] = d

In [40]:
for date in tqdm(df_h_regional.date):
    set_train_type_desc_regional(date) 

  0%|          | 0/24 [00:00<?, ?it/s]/home/watts/anaconda3/envs/rss/lib/python3.5/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/watts/anaconda3/envs/rss/lib/python3.5/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
100%|██████████| 24/24 [20:18<00:00, 50.77s/it]


In [41]:
df_train.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable,city,...,store_type,cluster,dcoilwtico,dom,mon,dow,doy,dcount,type,description
0,0,2013-01-01,25,103665,7.0,Unk,BREAD/BAKERY,2712,1,Salinas,...,D,1,93.14,1,1,1,1,734869,Holiday,Primer dia del ano
1,1,2013-01-01,25,105574,1.0,Unk,GROCERY I,1045,0,Salinas,...,D,1,93.14,1,1,1,1,734869,Holiday,Primer dia del ano
2,2,2013-01-01,25,105575,2.0,Unk,GROCERY I,1045,0,Salinas,...,D,1,93.14,1,1,1,1,734869,Holiday,Primer dia del ano
3,3,2013-01-01,25,108079,1.0,Unk,GROCERY I,1030,0,Salinas,...,D,1,93.14,1,1,1,1,734869,Holiday,Primer dia del ano
4,4,2013-01-01,25,108701,1.0,Unk,DELI,2644,1,Salinas,...,D,1,93.14,1,1,1,1,734869,Holiday,Primer dia del ano


In [42]:
df_train.shape

(125497040, 21)

In [43]:
# df_train9 = df_train

In [44]:
# df_train9 = df_train9.reset_index(drop=True)

In [45]:
# df_train9.shape

In [46]:
# feather.write_dataframe(df_train9, '../cache/train9.feather')

In [47]:
df_train['pd'] = 0 # payday

In [48]:
df_train.pd[(df_train.dom == 15)]= 1

/home/watts/anaconda3/envs/rss/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [49]:
df_train.pd[(df_train.dom == 31) & (df_train.mon == 1)]= 1
df_train.pd[(df_train.dom == 31) & (df_train.mon == 3)]= 1
df_train.pd[(df_train.dom == 30) & (df_train.mon == 4)]= 1
df_train.pd[(df_train.dom == 31) & (df_train.mon == 5)]= 1
df_train.pd[(df_train.dom == 30) & (df_train.mon == 6)]= 1
df_train.pd[(df_train.dom == 31) & (df_train.mon == 7)]= 1
df_train.pd[(df_train.dom == 31) & (df_train.mon == 8)]= 1
df_train.pd[(df_train.dom == 30) & (df_train.mon == 9)]= 1
df_train.pd[(df_train.dom == 31) & (df_train.mon == 10)]= 1
df_train.pd[(df_train.dom == 30) & (df_train.mon == 11)]= 1
df_train.pd[(df_train.dom == 31) & (df_train.mon == 12)]= 1

/home/watts/anaconda3/envs/rss/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/watts/anaconda3/envs/rss/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/watts/anaconda3/envs/rss/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we c

In [50]:
df_train.pd[df_train.date == '2013-02-28']= 1
df_train.pd[df_train.date == '2014-02-28']= 1
df_train.pd[df_train.date == '2015-02-28']= 1
df_train.pd[df_train.date == '2016-02-29']= 1
df_train.pd[df_train.date == '2017-02-28']= 1

/home/watts/anaconda3/envs/rss/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/watts/anaconda3/envs/rss/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/watts/anaconda3/envs/rss/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we c

In [51]:
df_train.shape

(125497040, 22)

In [52]:
df_train.isnull().sum()

id             0
date           0
store_nbr      0
item_nbr       0
unit_sales     0
onpromotion    0
family         0
class          0
perishable     0
city           0
state          0
store_type     0
cluster        0
dcoilwtico     0
dom            0
mon            0
dow            0
doy            0
dcount         0
type           0
description    0
pd             0
dtype: int64

In [53]:
np.datetime64('2016-04-16') - np.datetime64('2013-01-01')

numpy.timedelta64(1201,'D')

In [54]:
(np.datetime64('2016-04-16') - np.datetime64('2016-04-08'))/np.timedelta64(7, 'D')

1.1428571428571428

In [55]:
int(np.ceil((np.datetime64('2016-04-16') - np.datetime64('2016-04-08'))/np.timedelta64(7, 'D')))

2

In [56]:
def weeks_before_earthquake(date):
    if np.datetime64('2016-04-16') <= np.datetime64(date):
        return 0
    return int(np.ceil((np.datetime64('2016-04-16') - np.datetime64(date))/np.timedelta64(7, 'D')))

In [57]:
def weeks_after_earthquake(date):
    if np.datetime64('2016-04-16') >= np.datetime64(date):
        return 0
    return int(np.ceil((np.datetime64(date) - np.datetime64('2016-04-16'))/np.timedelta64(7, 'D')))

In [58]:
df_train['wbe'] = 0
df_train['wae'] = 0

In [59]:
df_train['wbe'] = df_train['date'].map(lambda d: weeks_before_earthquake(d))

In [60]:
df_train['wae'] = df_train['date'].map(lambda d: weeks_after_earthquake(d))

In [61]:
df_train.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable,city,...,dom,mon,dow,doy,dcount,type,description,pd,wbe,wae
0,0,2013-01-01,25,103665,7.0,Unk,BREAD/BAKERY,2712,1,Salinas,...,1,1,1,1,734869,Holiday,Primer dia del ano,0,172,0
1,1,2013-01-01,25,105574,1.0,Unk,GROCERY I,1045,0,Salinas,...,1,1,1,1,734869,Holiday,Primer dia del ano,0,172,0
2,2,2013-01-01,25,105575,2.0,Unk,GROCERY I,1045,0,Salinas,...,1,1,1,1,734869,Holiday,Primer dia del ano,0,172,0
3,3,2013-01-01,25,108079,1.0,Unk,GROCERY I,1030,0,Salinas,...,1,1,1,1,734869,Holiday,Primer dia del ano,0,172,0
4,4,2013-01-01,25,108701,1.0,Unk,DELI,2644,1,Salinas,...,1,1,1,1,734869,Holiday,Primer dia del ano,0,172,0


In [62]:
df_train[df_train.date == '2016-04-16'].shape

(100966, 24)

In [63]:
df_train[df_train.date == '2016-04-16'].head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable,city,...,dom,mon,dow,doy,dcount,type,description,pd,wbe,wae
76410846,76410846,2016-04-16,1,103665,4.0,False,BREAD/BAKERY,2712,1,Quito,...,16,4,5,107,736070,Event,Terremoto Manabi,0,0,0
76410847,76410847,2016-04-16,1,105574,11.0,False,GROCERY I,1045,0,Quito,...,16,4,5,107,736070,Event,Terremoto Manabi,0,0,0
76410848,76410848,2016-04-16,1,105575,7.0,False,GROCERY I,1045,0,Quito,...,16,4,5,107,736070,Event,Terremoto Manabi,0,0,0
76410849,76410849,2016-04-16,1,105577,3.0,False,GROCERY I,1045,0,Quito,...,16,4,5,107,736070,Event,Terremoto Manabi,0,0,0
76410850,76410850,2016-04-16,1,105737,3.0,False,GROCERY I,1044,0,Quito,...,16,4,5,107,736070,Event,Terremoto Manabi,0,0,0


In [64]:
df_train[df_train.date == '2016-04-17'].head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable,city,...,dom,mon,dow,doy,dcount,type,description,pd,wbe,wae
76511812,76511812,2016-04-17,1,105574,32.0,False,GROCERY I,1045,0,Quito,...,17,4,6,108,736071,Event,Terremoto Manabi+1,0,0,1
76511813,76511813,2016-04-17,1,105575,6.0,False,GROCERY I,1045,0,Quito,...,17,4,6,108,736071,Event,Terremoto Manabi+1,0,0,1
76511814,76511814,2016-04-17,1,105577,12.0,False,GROCERY I,1045,0,Quito,...,17,4,6,108,736071,Event,Terremoto Manabi+1,0,0,1
76511815,76511815,2016-04-17,1,105737,6.0,False,GROCERY I,1044,0,Quito,...,17,4,6,108,736071,Event,Terremoto Manabi+1,0,0,1
76511816,76511816,2016-04-17,1,105857,1.0,False,GROCERY I,1092,0,Quito,...,17,4,6,108,736071,Event,Terremoto Manabi+1,0,0,1


In [65]:
df_train[df_train.date == '2016-04-15'].head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable,city,...,dom,mon,dow,doy,dcount,type,description,pd,wbe,wae
76316906,76316906,2016-04-15,1,103665,4.0,False,BREAD/BAKERY,2712,1,Quito,...,15,4,4,106,736069,Work Day,Normal,1,1,0
76316907,76316907,2016-04-15,1,105574,3.0,False,GROCERY I,1045,0,Quito,...,15,4,4,106,736069,Work Day,Normal,1,1,0
76316908,76316908,2016-04-15,1,105575,13.0,False,GROCERY I,1045,0,Quito,...,15,4,4,106,736069,Work Day,Normal,1,1,0
76316909,76316909,2016-04-15,1,105577,2.0,False,GROCERY I,1045,0,Quito,...,15,4,4,106,736069,Work Day,Normal,1,1,0
76316910,76316910,2016-04-15,1,105857,13.0,False,GROCERY I,1092,0,Quito,...,15,4,4,106,736069,Work Day,Normal,1,1,0


In [66]:
# weeks from earthquake
df_train['wfe'] = df_train['wbe'].astype(int) + df_train['wae'].astype(int)

In [67]:
df_train.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable,city,...,mon,dow,doy,dcount,type,description,pd,wbe,wae,wfe
0,0,2013-01-01,25,103665,7.0,Unk,BREAD/BAKERY,2712,1,Salinas,...,1,1,1,734869,Holiday,Primer dia del ano,0,172,0,172
1,1,2013-01-01,25,105574,1.0,Unk,GROCERY I,1045,0,Salinas,...,1,1,1,734869,Holiday,Primer dia del ano,0,172,0,172
2,2,2013-01-01,25,105575,2.0,Unk,GROCERY I,1045,0,Salinas,...,1,1,1,734869,Holiday,Primer dia del ano,0,172,0,172
3,3,2013-01-01,25,108079,1.0,Unk,GROCERY I,1030,0,Salinas,...,1,1,1,734869,Holiday,Primer dia del ano,0,172,0,172
4,4,2013-01-01,25,108701,1.0,Unk,DELI,2644,1,Salinas,...,1,1,1,734869,Holiday,Primer dia del ano,0,172,0,172


In [76]:
df_train.shape

(125497040, 25)

In [68]:
feather.write_dataframe(df_train, '../cache/train10_nt.feather')

In [69]:
# create a sample data frame

In [70]:
df_10 = df_train.sample(frac=0.1, replace=False)

In [71]:
df_10.shape

(12549704, 25)

In [72]:
feather.write_dataframe(df_10, '../cache/train10_sample_10_nt.feather')

In [73]:
df_1 = df_train.sample(frac=0.01, replace=False)

In [74]:
df_1.shape

(1254970, 25)

In [75]:
feather.write_dataframe(df_1, '../cache/train10_sample_1_nt.feather')